# Step 1. Data extraction

In [3]:
import pandas as pd

df = pd.read_csv("db_table.csv")

,date,updateDate,code1,code2,code3,value,source
0,2017-01-01,2022-01-19 12:00:00,shC,CA,PROD,160.0,S1
1,2017-01-01,2022-04-30 18:00:00,shC,CA,PROD,200.0,S1
2,2017-01-01,2022-01-19 12:00:00,shC,CA,PROD,160.0,S1
3,2010-01-01,2010-12-31 00:00:00,shAAz,W/ALL/DUR,AH,6200.0,S1
4,2010-01-01,2010-12-31 00:00:00,shAAz,W/ALL/DUR,AP,3700.0,S1
...,...,...,...,...,...,...,...
9998,2010-01-01,2010-12-01 00:00:00,shAIa,S/1,STKS/ON_FARM,160000000.0,S1
9999,2010-01-01,2010-12-31 00:00:00,shAVa,C/1,YLD,76.9,S1
10000,2010-01-01,2010-12-31 00:00:00,shAVa,C/1,AP,3900.0,S1
10001,2010-01-01,2010-12-31 00:00:00,shAVa,C/1,YLD,26.3,S1


# Step 2. Data transformation

Creating "bitCode" column:
* 1 - if source equals S1, 
* 0 otherwise.

In [5]:
def categorize_source(source):
    if source == "S1":
        return 1
    return 0

df["bitCode"] = df["source"].apply(categorize_source)

Creating "siCode" column:
* A - if bitCode equals 1 and code3 is AP or AH, 
* B - if bitCode equals 1 and code3 is PRD, 
* BpA - if bitCode equals 1 and code3 is YLD, 
* H - if bitCode equals 0 and code3 is AP or AH, 
* T - if bitCode equals 0 and code3 is PRD, 
* TpH - if bitCode equals 0 and code3 is YLD, 
* None otherwise. 

In [8]:
si_code_mapping = {
    (1, "AP"): "A",
    (1, "AH"): "A",
    (1, "PRD"): "BpA",
    (1, "YLD"): "B",
    (0, "AP"): "H",
    (0, "AH"): "H",
    (0, "PRD"): "TpH",
    (0, "YLD"): "T"
}

df["siCode"] = (
    df[["bitCode", "code3"]]
    .apply(
        lambda row: si_code_mapping.get((row["bitCode"], row["code3"]), None), axis=1
    )
)

0        None
1        None
2        None
3           A
4           A
         ... 
9998     None
9999        B
10000       A
10001       B
10002    None
Name: siCode, Length: 10003, dtype: object

# Step 3. Data loading
Preparation for data transform to frontend

array(['S1'], dtype=object)